In [33]:
import math
import os

import numpy as np
import pandas as pd

In [10]:
keywords = set('''drink drinker drinks drinking drank wine champgne alcohol alcoholics alcoholism beer beers bottle bottles
    pint pints cocktail cocktails bar brewery lounge pub liquor booze vodka tequila gin ciroc margarita margaritas shot shots
    ale whiskey lager tipsy drunk sober wasted pregame pregaming'''.split())

def prefilter(text):
    return bool(set(text.split()).intersection(keywords))

In [18]:
df = pd.read_csv('E:/september blocked/remaining_tweets_0.csv')
df2 = pd.read_csv('E:/september untarred/twitter_2015-09-01_00h.json.tar.gz0.csv')

C:\Users\Tom Work\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (0,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [22]:
df2.head()

,text,id,created_at
0,Everyone going to philly don't rush it it's go...,638561918433759232,Tue Sep 01 03:59:59 +0000 2015
1,If she ain't tryin get it she can get dropped ...,638561918148440064,Tue Sep 01 03:59:59 +0000 2015
2,Was i the only one who thought gohan was cute?...,638561918123270144,Tue Sep 01 03:59:59 +0000 2015
3,@TungguSekejap bila doh!? Aku kat ny skunk. Huhu,638561918618312704,Tue Sep 01 03:59:59 +0000 2015
4,@IamHolleman we still need to hit up that new ...,638561918660288512,Tue Sep 01 03:59:59 +0000 2015


In [29]:
df.to_csv('E:/test.csv', encoding='utf8')

Split uniformly by number of tweets instead of by time

In [32]:
%%time

tweets = pd.DataFrame()
file_num = 0
for file in os.listdir('E:/september untarred'):
    _df = pd.read_csv('E:/september untarred/'  + file)
    tweets = pd.concat([tweets, _df])
    if len(tweets) > 500000:
        tweets[:500000].to_csv('E:/september blocked/tweets_' + str(file_num) +'.csv', encoding='utf8')
        tweets = tweets[500000:]
        file_num += 1
tweets.to_csv('E:/september blocked/tweets_' + str(file_num) +'.csv')

Wall time: 27min 12s


Split into drinking an nondrinking by keyword

In [11]:
%%time
drinking = pd.DataFrame()
i = 0
for file in os.listdir('E:/september blocked')[2:]:
    try:
        _df = pd.read_csv('E:/september blocked/' + file, encoding='utf8', engine='python')
        drinking = pd.concat([drinking, _df[_df.text.apply(str).apply(prefilter)]])
        drinking['id'].apply(str)
        if len(drinking) > 500000:
            drinking[:500000].to_csv('E:/september blocked/drinking/drinking_' + str(i) +'.csv', encoding='utf8')
            drinking = drinking[500000:]
            i+=1
    except Exception as e:
        print (file)
        print (e)
drinking.to_csv('E:/september blocked/drinking2/drinking_' + str(i) +'.csv', encoding='utf8')

nondrinking
[Errno 13] Permission denied: 'E:/september blocked/nondrinking'
remaining_tweets_136.csv
'utf-8' codec can't decode byte 0x85 in position 395: invalid start byte
Wall time: 26min 28s


In [21]:
drinking = pd.read_csv('E:/september blocked/drinking/drinking_0.csv', dtype={'id': str})

In [13]:
drinking.shape

(498303, 5)

In [23]:
drinking['random_number'] = drinking.text.apply(hash)

In [22]:
(drinking.id[0])

'638561918660288512'

In [24]:
drinking.index = drinking.id

In [26]:
drinking.head()

,Unnamed: 0,Unnamed: 0.1,text,id,created_at,random_number
id,,,,,,
638561918660288512,4,4,@IamHolleman we still need to hit up that new ...,638561918660288512,Tue Sep 01 03:59:59 +0000 2015,4020266428001935890
638561921806024704,39,39,@young_zamp we aren't drinking lol,638561921806024704,Tue Sep 01 04:00:00 +0000 2015,3871613784586990036
638561941477322753,255,255,@Janaeethomas idk if it works or not but I scr...,638561941477322753,Tue Sep 01 04:00:05 +0000 2015,-4684982304355642440
638561967897231360,537,537,I'm feeling tipsy and not in the good way,638561967897231360,Tue Sep 01 04:00:11 +0000 2015,5214561322629894300
638562056208183296,1491,1491,wine gone knock me out everytime 😅 I been slee...,638562056208183296,Tue Sep 01 04:00:32 +0000 2015,1935074228051928726


In [27]:
pickle.dump(drinking, open('pickles/sept_drinking.p', 'wb'))

In [28]:
import os
import datetime
def make_batch(pickled_df, size, output_dir):
    """
    pickled_df: unlabeled data with 'random_number' column
    size: # of items batch to be created
    output_dir: where the outputs are saved
    """
    # Load data
    X = pickle.load(open(pickled_df, 'rb'))
    
    # select size subset
    batch = X.sort_values(by='random_number').head(size)
    
    # process for turk
    batch["text"] = batch.text.str.encode("utf-8")
    date = str(datetime.date.today())
    time = datetime.datetime.now().time()
    key = batch.iloc[0].random_number
    
    # persist
    description = "/c_amt {} {}h{}m {}".format(date, time.hour, time.minute, key)
    if not os.path.exists(output_dir + description):
        os.makedirs(output_dir + description)
    batch[["text", "random_number"]].to_csv(output_dir + description +  description + '.csv')
    pickle.dump(batch, open(output_dir + description + description +".p", 'wb'))
    X = X.drop(batch.index)
    pickle.dump(X, open(pickled_df, 'wb'))

In [29]:
make_batch('pickles/sept_drinking.p', 1000, 'amt/sept')

In [32]:
import pickle
from classification.prediction import PredictionTransformer

In [34]:
clf_alc = pickle.load(open('pickles/clf_alc_UPDATED.p', 'rb'))
clf_fpa = pickle.load(open('pickles/clf_fpa_UPDATED.p', 'rb'))
clf_fpl = pickle.load(open('pickles/clf_fpl_double_labeled', 'rb'))
clf = PredictionTransformer(clf_alc, clf_fpa, clf_fpl)

In [35]:
%%time
labeled = clf(drinking)

Wall time: 2h 8min 23s


In [36]:
%%time
labeled.index = pd.to_datetime(labeled.created_at)

Wall time: 5min 3s


In [38]:
pickle.dump(labeled, open('pickles/sept_drinking_labeled.p', 'wb'))